## Imports

In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

## Trigger request to download data

In [2]:
article_id = "14096681"
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figsharerainfall/"
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  
files = data["files"]            

## Create zip file with downloaded data

In [4]:
%%time
files_to_dl = ["data.zip"]
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: total: 4.61 s
Wall time: 3min 54s


## Extract data into output directory

In [5]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: total: 20.4 s
Wall time: 35.7 s


## Combining data CSVs

In [27]:
%%time
import pandas as pd
excluded_files = ["figsharerainfall\\observed_daily_rainfall_SYD.csv"]
files = glob.glob('figsharerainfall/*.csv')
files = list(set(files) - set(excluded_files))
df = pd.concat((pd.read_csv(file, index_col=0)
                .assign(model=re.findall("/([^_]*)", file)[0])
                for file in files)
              )
df.to_csv("figsharerainfall/combined_data.csv")

CPU times: total: 7min 6s
Wall time: 12min 49s


| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Natalie    |                  |     |           |        |            |
| Nate    |                  |     |           |        |            |
| Nikita    |    Windows              |  16GB   |   12th Gen Intel(R) Core(TM) i7-1255U | yes |   12min 49s         |
| Stephen    |                  |     |           |        |            |

## Perform Simple EDA

| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Natalie    |                  |     |           |        |            |
| Nate    |                  |     |           |        |            |
| Nikita    |    Windows              | 16GB    |  12th Gen Intel(R) Core(TM) i7-1255U  |  yes   |            |
| Stephen    |                  |     |           |        |            |

## Perform Simple EDA in R